In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import pandas as pd

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)  # leave browser open when done
driver = webdriver.Chrome(options=options)

try:
    # --- 2) Go to login page & log in ---
    driver.get("https://islands.smp.uq.edu.au/login.php")
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.NAME, "email")))
    driver.find_element(By.NAME, "email").send_keys("matthew.stoebe@colostate.edu")
    driver.find_element(By.NAME, "word").send_keys("TheIslands")
    driver.find_element(By.CSS_SELECTOR, "input[type='submit'][value='Sign In']").click()
    
    driver.get("https://islands.smp.uq.edu.au/project.php")
    
    cards = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.contact")))
    
    contacts = []
    for card in cards:
        a = card.find_element(By.CSS_SELECTOR, "div.name a")
        name = a.text.replace("\n", " ")
        profile_url = urljoin(driver.current_url, a.get_attribute("href"))
        contacts.append({"name": name, "profile_url": profile_url})
    
finally:
    # driver.quit()                       # uncomment to close browser automatically
    pass


In [6]:
print(contacts)


[{'name': 'Akane Abels', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=tvr3smcydj'}, {'name': 'Leon Bager', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=ta43k375tr'}, {'name': 'Paul Bager', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=zlgyztmrxn'}, {'name': 'Nihal Bahadur', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=u7c7hptsqp'}, {'name': 'Britt Blomgren', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=679tawpk3w'}, {'name': 'Elias Blomgren', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=ryhv983pr8'}, {'name': 'Tobias Blomgren', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=5qb9p989sp'}, {'name': 'Franziska Brandt', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=eymh53yuzc'}, {'name': 'Jasmin Brandt', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?id=ubv5v7mtz9'}, {'name': 'Ross Brown', 'profile_url': 'https://islands.smp.uq.edu.au/islander.php?